In [111]:
import csv
import pandas as pd
import os
import json
from tqdm import tqdm
from gensim.models import Word2Vec
import numpy as np
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr, pointbiserialr
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, validation_curve, GridSearchCV, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, make_scorer
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from category_encoders import TargetEncoder
import matplotlib.pyplot as plt
from itertools import product
from pprint import pprint
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler
from numpy import array  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re2 as re

import dask.dataframe as df
from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore', message='A NumPy ')

In [8]:
# all_df = pd.read_csv('/data/kiho/webtrack/worldwideweb/all_df_350.csv', index_col=0)
feat_ad=pd.read_csv('/data/kiho/webtrack/feature_extracted/adgraph/ad_features.csv',index_col=0)
label_ad=pd.read_csv('/data/kiho/webtrack/feature_extracted/adgraph/ad_labelled.csv',index_col=0)
feat_web=pd.read_csv('/data/kiho/webtrack/feature_extracted/webgraph/web_features.csv',index_col=0)
label_web=pd.read_csv('/data/kiho/webtrack/feature_extracted/webgraph/web_labelled.csv',index_col=0)

In [11]:
df_labelled_ad = feat_ad.merge(label_ad[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled_ad = df_labelled_ad[df_labelled_ad['label'] != "Error"]

df_labelled_web = feat_web.merge(label_web[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled_web = df_labelled_web[df_labelled_web['label'] != "Error"]

df_word_embed = feat_ad.merge(label_ad[['visit_id', 'name', 'label', 'top_level_url']], on=['visit_id', 'name'])
df_word_embed = df_word_embed[df_word_embed['label'] != "Error"]
df_word_embed = df_word_embed[['visit_id', 'name', 'top_level_url', 'label']]

/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1210: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [126]:
df_word_embed['top_level_url'] = df_word_embed['top_level_url'].str.replace("http://", "").str.replace("https://", "")
df_word_embed['top_level_url'] = df_word_embed['top_level_url'].str.replace("[", "%5B").str.replace("]", "%5D")
train, test = train_test_split(df_word_embed, test_size=0.2, random_state=42)

/tmp/ipykernel_6278/1649624601.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_word_embed['top_level_url'] = df_word_embed['top_level_url'].str.replace("[", "%5B").str.replace("]", "%5D")


In [127]:
ad_col = df_labelled_ad.columns
web_col = df_labelled_web.columns
col_list = []

for i in range(len(ad_col)):
    for j in range(len(web_col)):
        if ad_col[i] == web_col[j]:
            col_list.append(ad_col[i])
# col_list = col_list[2:-1]
df_outer_web = df_labelled_web.copy().drop(col_list, axis=1)
integ_df = pd.concat([df_labelled_ad.drop('label', axis=1), df_outer_web, df_labelled_ad['label']], axis=1)    ### AdGraph, WebGraph Features

In [320]:
def genFeatures(charlist, char_vec, vocabs):
    UrlVec = np.average([char_vec[vocabs[c]] for c in charlist], axis=0)
    return UrlVec

def wordFeature(url_df, test_df, _istarg):
    
    features = []
    features_test = []
    vector_size=30
    if _istarg:
        vector_size=200
    
    print('model load')
    model = Word2Vec(url_df, vector_size=vector_size, window=3, min_count=1, workers=30, hs=1, sg=1)

    b_word_vectors = model.wv
    b_vocabs = b_word_vectors.key_to_index
    b_char_vectors = [b_word_vectors[v] for v in b_vocabs]

    for j in tqdm(range(len(url_df))):
        features.append(list(genFeatures(url_df[j], b_char_vectors, b_vocabs)))
        
    for j in tqdm(range(len(test_df))):
        features_test.append(list(genFeatures(test_df[j], b_char_vectors, b_vocabs)))
    return pd.DataFrame(features), pd.DataFrame(features_test)

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [186]:
_istarget=True
target_word_ft, target_word_ft_test = wordFeature(list(train['name']), list(test['name']), _istarget)
_istarget=False
source_word_ft, source_word_ft_test = wordFeature(list(train['top_level_url']), list(test['top_level_url']),_istarget)

fqdn_col = []
for i in range(0, 30):
    fqdn_col.append("fqdn_" + str(i))

source_word_ft.columns = fqdn_col
source_word_ft_test.columns = fqdn_col
    
req_col = []
for i in range(0, 200):
    req_col.append("req_" + str(i))
    
target_word_ft.columns = req_col
target_word_ft_test.columns = req_col

train_url230_sg = pd.concat([source_word_ft, target_word_ft], axis=1)
test_url230_sg = pd.concat([source_word_ft_test, target_word_ft_test], axis=1)

model load


100%|██████████| 166032/166032 [00:20<00:00, 8173.21it/s]


model load


100%|██████████| 166032/166032 [00:05<00:00, 29773.19it/s]


In [235]:
all_df_883 = pd.read_csv('all_df_883.csv', index_col=0)

In [259]:
new_features = all_df_883[all_df_883.columns[303:]].copy()
new_features.drop(['contentname', 'label'], axis=1, inplace=True)
new_features.fillna(0, inplace=True)

In [187]:
# train_url230_sg.to_csv('train_url230_sg.csv')
# test_url230_sg.to_csv('test_url230_sg.csv')

In [329]:
ngram_train, ngram_test = train_test_split(new_features, test_size=0.2, random_state=42)
ngram_train.reset_index(drop=True, inplace=True)
ngram_test.reset_index(drop=True, inplace=True)

integ_train, integ_test = train_test_split(integ_df, test_size=0.2, random_state=42)
integ_train.reset_index(drop=True, inplace=True)
integ_test.reset_index(drop=True, inplace=True)

train_df = pd.concat([integ_train[integ_train.columns[:-1]], train_url230_sg, ngram_train, integ_train['label']], axis=1)
test_df = pd.concat([integ_test[integ_test.columns[:-1]], test_url230_sg, ngram_test, integ_test['label']], axis=1)

In [4]:
sum_resolve = pd.read_csv('/data/kiho/webtrack/worldwideweb/sum_resolve.csv', index_col=0)

In [9]:
train_resolve, test_resolve = train_test_split(sum_resolve, test_size=0.2, random_state=42)
train_resolve.reset_index(drop=True, inplace=True)
test_resolve.reset_index(drop=True, inplace=True)

In [2]:
train_df = pd.read_csv('/data/kiho/webtrack/worldwideweb/all_df_883_train.csv', index_col=0)
test_df = pd.read_csv('/data/kiho/webtrack/worldwideweb/all_df_883_test.csv', index_col=0)

In [11]:
train_df.loc[train_resolve.index, train_resolve.columns] = train_resolve
test_df.loc[test_resolve.index, test_resolve.columns] = test_resolve

In [14]:
# avg_indent = train_df['avg_ident'] ** -1
# for i in range(len(avg_indent)):
#     if avg_indent[i] == np.inf:
#         avg_indent[i] = 0
# train_df['avg_ident'] = avg_indent

# avg_indent = test_df['avg_ident'] ** -1
# for i in range(len(avg_indent)):
#     if avg_indent[i] == np.inf:
#         avg_indent[i] = 0
# test_df['avg_ident'] = avg_indent

train_df.to_csv('all_df_883_train.csv')
test_df.to_csv('all_df_883_test.csv')

In [15]:
Te = TargetEncoder()
Te.fit(train_df.content_policy_type, train_df.label)
train_df.content_policy_type = Te.transform(train_df.content_policy_type)
test_df.content_policy_type = Te.transform(test_df.content_policy_type)

In [16]:
cnt = 0
filtered_feat = []
for i in train_df.columns[2:-1]:
    correlation, p_value = pointbiserialr(train_df.label, train_df[i])
    if ((p_value > 0.1)):
        print(i)
        print('Correlation coefficient:', correlation)
        print('p-value:', p_value)
        cnt += 1
        filtered_feat.append(i)
train_df.drop(filtered_feat, inplace=True, axis=1)
print('Removed : ', cnt)

/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


num_get_url_src
Correlation coefficient: 0.0016470977375069842
p-value: 0.17950359916468822
fqdn_1
Correlation coefficient: 0.00014345295281208256
p-value: 0.9069350199404169
HTMLCanvasElement
Correlation coefficient: -0.0013909279472267468
p-value: 0.25699514345658464
AnalyserNode
Correlation coefficient: 0.0010123624541648548
p-value: 0.4093637498561586
OPTIONS
Correlation coefficient: 0.0015053914351010429
p-value: 0.219896711785617
HEAD
Correlation coefficient: -0.001414538418301978
p-value: 0.24900766014089892
PUT
Correlation coefficient: 0.001636748542036324
p-value: 0.18225268024245003
order
Correlation coefficient: -0.0019180809239860187
p-value: 0.1180243015840343
ng_0_0_10
Correlation coefficient: -0.0011902695977293299
p-value: 0.3320477857722406
ng_0_0_14
Correlation coefficient: 0.00040508963768943194
p-value: 0.7413067693666847
ng_0_1_3
Correlation coefficient: 0.0006271882828025957
p-value: 0.6092668831172424
ng_0_2_1
Correlation coefficient: -0.0015049328550088498
p-val

In [17]:
X = train_df.drop(['visit_id', 'name', 'label'], axis=1)
y = train_df.label

In [24]:
X = X.loc[:len(X)/2]
y = y.loc[:len(y)/2]

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42) # train / vaild
# Train a RandomForest model
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": importances})
print("feature_importances")
# Get permutation importances
result = permutation_importance(clf, X_valid, y_valid, n_repeats=5, random_state=42, n_jobs=-1)
print("result")
# Convert the importances into a DataFrame
perm_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": result.importances_mean})

# Print permutation importances
print(perm_importances)

feature_importances.columns = ['feature', 'RFI_importance']
perm_importances.columns = ['feature', 'PI_importance']
concat = feature_importances.copy()
concat = pd.concat([concat, perm_importances.PI_importance], axis=1)
concat['RFI_PI_MEAN'] = (concat.RFI_importance + concat.PI_importance) / 2
concat
concat.to_csv('RFI_PI_1006.csv')

feature_importances


/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this vers

result
                 feature  importance
0    content_policy_type    0.001599
1             url_length    0.001424
2           is_subdomain    0.009829
3            is_valid_qs    0.000000
4         is_third_party    0.006484
..                   ...         ...
692            ast_depth    0.000214
693          ast_breadth    0.000304
694            avg_ident    0.000145
695      avg_charperline    0.000437
696         brackettodot    0.000325

[697 rows x 2 columns]


In [94]:
graphFeatures=[  # 52
    "num_nodes",
    "num_edges",
    "nodes_div_by_edges",
    "edges_div_by_nodes",
    "in_degree",
    "out_degree",
    "in_out_degree",
    "average_degree_connectivity",
    "is_ancestor_script",
    "ascendant_has_ad_keyword",
    "descendant_of_eval_or_function",
    "ascendant_script_has_eval_or_function",
    "ascendant_script_has_fp_keyword",
    "ascendant_script_length",
    "ancestors",
    "descendants",
    "closeness_centrality",
    "eccentricity",
    "num_script_predecessors",
    "num_script_successors",
    "num_requests_received",
    "num_redirects_sent",
    "num_redirects_rec",
    "max_depth_redirect",
    "indirect_in_degree",
    "indirect_out_degree",
    "indirect_ancestors",
    "indirect_descendants",
    "indirect_closeness_centrality",
    "indirect_average_degree_connectivity",
    "indirect_eccentricity",
    "indirect_mean_in_weights",
    "indirect_min_in_weights",
    "indirect_max_in_weights",
    "indirect_mean_out_weights",
    "indirect_min_out_weights",
    "indirect_max_out_weights",
    "num_set_get_src",
    "num_set_mod_src",
    "num_set_url_src",
    # "num_get_url_src",
    "num_set_get_dst",
    "num_set_mod_dst",
    "num_set_url_dst",
    "num_get_url_dst",
    "indirect_all_in_degree",
    "indirect_all_out_degree",
    "indirect_all_ancestors",
    "indirect_all_descendants",
    "indirect_all_closeness_centrality",
    "indirect_all_average_degree_connectivity",
    "indirect_all_eccentricity"
]

pi_is_zero = perm_importances.loc[(perm_importances.PI_importance <= 0)].feature.tolist()



In [95]:
X_wo_graph = X.drop(graphFeatures + pi_is_zero, axis = 1)

In [96]:
X_wo_graph

,content_policy_type,url_length,is_subdomain,is_third_party,base_domain_in_query,ad_size_present,keyword_raw_present,keyword_char_present,is_parent_script,num_get_storage,...,ng_15_15_9,ng_15_15_11,ng_15_15_15,ng_18_14_0,ng_18_15_15,ast_depth,ast_breadth,avg_ident,avg_charperline,brackettodot
0,0.475045,66,1,0,0,0,0,0,0,4,...,0.005565,0.0,0.065665,0.0,0.0,82.0,8638.0,0.308177,34650.000000,2.802360
1,0.079060,68,1,0,0,0,0,0,0,4,...,0.001343,0.0,0.077839,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.475045,49,0,1,0,0,1,1,1,1101,...,0.005828,0.0,0.061511,0.0,0.0,84.0,46677.0,0.252925,652.146739,3.153756
3,0.504457,1724,0,1,0,1,1,1,1,2,...,0.000000,0.0,0.049685,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.384621,76,0,1,0,0,0,0,0,0,...,0.000000,0.0,0.063516,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332060,0.384621,116,0,1,0,0,1,0,1,4,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
332061,0.384621,122,1,0,0,0,1,1,0,11,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
332062,0.384621,64,0,1,0,0,0,0,0,4,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
332063,0.035995,29,1,0,0,0,0,0,0,0,...,0.000644,0.0,0.007088,0.0,0.0,31.0,1965.0,0.214286,168.000000,2.000000


In [104]:
X_train, X_valid, y_train, y_valid = train_test_split(X_wo_graph, y, shuffle=True, test_size=0.2, random_state=42) # train / vaild

In [113]:
# rf = RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1)
# rf.fit(X_train, y_train)
# print("rf.fit(X_train, y_train)")
# result = permutation_importance(rf, X_train, y_train)
# perm_importance = result.importances_mean.argsort()

# rfi_pi_mean = concat['RFI_PI_MEAN'].copy()

min_features_to_select = 10
step = 1
selector = RFECV(clf, step=step, cv=5, min_features_to_select=min_features_to_select, scoring='accuracy', n_jobs=-1)
print("RFECV")

selector = selector.fit(X_wo_graph, y)
print("selector")

rfecv_support = selector.support_
rfecv_ranking = selector.ranking_
print("RFECV support:", rfecv_support)
print("RFECV ranking:", rfecv_ranking)
selected_perm_importance = perm_importances[rfecv_support]
print("Permutation importance for the selected features by RFECV:", selected_perm_importances)

RFECV


/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this vers

In [ ]:
res = []
for i in range(len(temp3.columns)):
    for j in range(i, len(temp3.columns)):
        if pearsonr(temp3[temp3.columns[i]], temp3[temp3.columns[j]]).pvalue >= 0.05:
            print(f"{temp3.columns[i]}, {temp3.columns[j]} : {pearsonr(temp3[temp3.columns[i]], temp3[temp3.columns[j]]).pvalue}")

In [ ]:
for i in range(len(temp3.columns)):
    for j in range(i, len(temp3.columns)):
        if spearmanr(temp3[temp3.columns[i]], temp3[temp3.columns[j]]).pvalue >= 0.05:
            print(f"{temp3.columns[i]}, {temp3.columns[j]} : {spearmanr(temp3[temp3.columns[i]], temp3[temp3.columns[j]]).pvalue}")